In [1]:
using CSV
using DataFrames

# Read CSV files
df = CSV.read("student_response.csv", DataFrame)
class_data = CSV.read("class_data_corrected.csv", DataFrame)
student_preferences = CSV.read("student_preferences_corrected.csv", DataFrame)

# Extract specific columns from the DataFrame
name_vals = df.Name
a_like = collect(df.attendance_likelihood)
head_volunteer = df.head_volunteer

# Drop unwanted columns
drop_cols = [
    :Name, :attendance_likelihood, :head_volunteer,
    :stay_all_day, :event, :specific_event, :event_1,
    :specific_event_1, Symbol("Which do you prefer?    =====================>"),
    :Event_grader, :Proctor, :Runner_Floater, :photographer
]

# Drop columns and remove the last column
select!(df, Not(drop_cols))

# Display remaining columns
println(name_vals)

# Create dictionaries for mappings
ind_to_event = Dict{Int, String}()
event_to_ind = Dict{String, Int}()

ind_to_name = Dict{Int, String}()
name_to_ind = Dict{String, Int}()

# Map indices to names
for (i, name) in enumerate(name_vals)
    ind_to_name[i] = name
    name_to_ind[name] = i
end

# Map indices to events
for (i, event) in enumerate(class_data.Event)
    ind_to_event[i] = event
    event_to_ind[event] = i
end

# Create student preferences dictionary
s_pref = Dict{Int, Vector{Int}}()

cols = string.(names(student_preferences)[2:end])  # Skip the "Name" column

for row in eachrow(student_preferences)
    name = row.Name
    name_ind = name_to_ind[name]
    # Get indices of preferred classes and map them to event indices
    class_inds = string.(findall(!iszero, row[2:end]))
    class_names = [event_to_ind[ind] for ind in class_inds]

    s_pref[name_ind] = class_names
end

# Print mappings (for debugging if needed)
println(ind_to_name)
println(ind_to_event)
println(s_pref)


String31["Ivan Ge", "Daniel Ochoa", "Alan Choi", "Jacob Mazumdar", "Sophie Wang", "Matthew Barnett", "Robin Xiong", "Anna Kaganov", "Jack Liu", "Lance Berkey", "Sabrina Meng", "Riddhi Bhagwat", "Amie Kitjasateanphun", "Isabella Duan", "Charlotte Myers", "Hector Flores", "April Wu", "Kennedy Gore", "Rafaa Qanash", "Maxi Attiogbe", "Davut Muhammetgulyyev", "Anna Liu", "Andrea Cosio", "Robin", "Abayo Joseph Desire", "Eric Delgado", "Sophia Lee", "Serena Pei", "Reina Wang", "Lama Alahdal ", "David Robichaud", "Triston Mohamed", "Allison Park", "Christopher Wang", "Jean Yu", "Macy Hogsett", "Matthew Nunez", "Audrey Lim", "Alana Yang", "Nicole Pardal", "Hanson He", "Claire Chen", "Richard Chen", "Pari Rajesh", "Felicity Zhou", "Katherine Li", "Wilson Zhu", "Sukrith Velmineti", "Alvin Zheng", "Caleb Zhao", "Vicky Yan", "Pragnya Govindu", "Kevin Toledo", "Ann Hashimoto ", "Esther Kinyanjui", "Muhammad Alnasser", "Rachel Jiang", "Richard Zhu", "Carol Jiang", "Hana Boulware", "Vernon Lin", "Mich

In [ ]:
using JuMP
using Gurobi

# Create a new model
model = Model(Gurobi.Optimizer)

num_students = size(df, 1)
num_times = 8
num_classes = 26

# Decision variables
vars = Dict{Tuple{Int, Int, Int, Int}, VariableRef}()

vars_to_minimize_availability_slack = []

for i in 1:num_students
    for j in 1:num_times
        if ismissing(df[i, j]) || df[i, j] == 0  # Handle missing or unavailable volunteer
            # If volunteer not available
            for k in 1:num_classes
                # Decision variables for p & np roles
                vars[(i, j, k, 0)] = @variable(model, binary = true)
                vars[(i, j, k, 1)] = @variable(model, binary = true)

                # Slack variables for unmet availability
                slack_1 = @variable(model)
                push!(vars_to_minimize_availability_slack, slack_1)
                @constraint(model, vars[(i, j, k, 1)] == 0)

                slack_0 = @variable(model)
                push!(vars_to_minimize_availability_slack, slack_0)
                @constraint(model, vars[(i, j, k, 0)] == 0)
            end
            continue
        end

        for k in 1:num_classes
            # Decision variables for available students
            vars[(i, j, k, 0)] = @variable(model, binary = true)
            vars[(i, j, k, 1)] = @variable(model, binary = true)

            # Principal or non-principal roles constraint
            @constraint(
                model,
                vars[(i, j, k, 0)] + vars[(i, j, k, 1)] <= 1,
            )

            # Non-head volunteers cannot be principal
            if head_volunteer[i] != 1
                @constraint(
                    model,
                    vars[(i, j, k, 1)] == 0,
                )
            end
        end
    end
end

# A person can only volunteer for one class at a time
for i in 1:num_students
    for j in 1:num_times
        @constraint(
            model,
            sum(vars[(i, j, k, p)] for k in 1:num_classes for p in 0:1) <= 1,
        )
    end
end

# Variables for slack constraints
vars_to_minimize_slack_p = []
vars_to_minimize_slack = []

# Demand constraints
for j in 1:num_times
    for k in 1:num_classes
        # Slack variable for unmet total demand
        slack = @variable(model)
        push!(vars_to_minimize_slack, slack)

        # Ensure total volunteer demand is met
        @constraint(
            model,
            sum(vars[(i, j, k, p)] for i in 1:num_students for p in 0:1) >= class_data.needed_volunteers[k] - slack,
        )

        # Ensure one principal volunteer per event
        slack_p = @variable(model)
        push!(vars_to_minimize_slack_p, slack_p)

        @constraint(
            model,
            sum(vars[(i, j, k, 1)] for i in 1:num_students) + slack_p == 1,
        )
    end
end

# Variables and constraints for continuity (penalizing discontinuities)
vars_to_minimize = []
vars_to_minimize_constr = []

for i in 1:num_students
    for j in 1:(num_times - 1)
        for k_1 in 1:num_classes
            for k_2 in 1:num_classes
                if k_1 == k_2
                    continue
                end
                for p_1 in 0:1
                    for p_2 in 0:1
                        diff = @variable(model, lower_bound = 0)

                        push!(vars_to_minimize, diff)

                        @constraint(model, diff >= vars[(i, j, k_1, p_1)] + vars[(i, j + 1, k_2, p_2)] - 1)
                    end
                end
            end
        end
    end
end

# Penalize assignments to non-preferred events
vars_to_minimize_pref = []
for i in 1:num_students
    student_pref = s_pref[i]
    for j in 1:num_times
        for k in 1:num_classes
            if k in student_pref
                continue
            end
            for p in 0:1
                push!(vars_to_minimize_pref, vars[(i, j, k, p)])
            end
        end
    end
end

# Set Objectives with priorities
@objective(
    model, Min,
    5 * sum(vars_to_minimize_slack) / length(vars_to_minimize_slack) +
    1 * sum(vars_to_minimize_slack_p) / length(vars_to_minimize_slack_p) +
    1 * sum(vars_to_minimize_pref) / length(vars_to_minimize_pref) +
    1 * sum(vars_to_minimize) / length(vars_to_minimize)
)
model.setParam('OutputFlag', 1)         # Enable output
model.setParam('LogToConsole', 1)      # Ensure console output
model.setParam('DisplayInterval', 1)   # Update every second

set_optimizer_attribute(model, "Threads", 8)  # Replace 8 with the desired number of cores

println("Model created.")
# Optimize the model
optimize!(model)

# Check if an optimal solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found.")
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found.")
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
_[56889]
_[56890]
_[56891]
_[56892]
_[56893]
_[56894]
_[56895]
_[56896]
_[56897]
_[56898]
_[56899]
_[56900]
_[56901]
_[56902]
_[56903]
_[56904]
_[56905]
_[56906]
_[56907]
_[56908]
_[56909]
_[56910]
_[56911]
_[56912]
_[56913]
_[56914]
_[56915]
_[56916]
_[56917]
_[56918]
_[56919]
_[56920]
_[56921]
_[56922]
_[56923]
_[56924]
_[56925]
_[56926]
_[56927]
_[56928]
_[56929]
_[56930]
_[56931]
_[56932]
_[56933]
_[56934]
_[56935]
_[56936]
_[56937]
_[56938]
_[56939]
_[56940]
_[56941]
_[56942]
_[56943]
_[56944]
_[56945]
_[56946]
_[56947]
_[56948]
_[56949]
_[56950]
_[56951]
_[56952]
_[56953]
_[56954]
_[56955]
_[56956]
_[56957]
_[56958]
_[56959]
_[56960]
_[56961]
_[56962]
_[56963]
_[56964]
_[56965]
_[56966]
_[56967]
_[56968]
_[56969]
_[56970]
_[56971]
_[56972]
_[56973]
_[56974]
_[56975]
_[56976]
_[56977]
_[56978]
_[56979]
_[56980]
_[56981]
_[56982]
_[56983]
_[56984]
_[56985]
_[56986]
_[56987]
_[56988]
_[56989]


Excessive output truncated after 524294 bytes.

_[112897]
_[112898]
_[112899]
_[112900]
_[112901]
_[112902]
_[112903]
_[112904]
_[112905]
_[112906]
_[112907]
_[112908]
_[112909]
_[112910]
_[112911]
_[112912]
_[112913]
_[112914]
_[112915]
_[112916]
_[112917]
_[112918]
_[112919]
_[112920]
_[112921]
_[112922]
_[112923]
_[112924]
_[112925]
_[112926]
_[112927]
_[112928]
_[112929]
_[112930]
_[112931]
_[112932]
_[112933]
_[112934]
_[112935]
_[112936]
_[112937]
_[112938]
_[112939]
_[112940]
_[112941]
_[112942]
_[112943]
_[112944]
_[112945]
_[112946]
_[112947]
_[112948]
_[112949]
_[112950]
_[112951]
_[112952]
_[112953]
_[112954]
_[112955]
_[112956]
_[112957]
_[112958]
_[112959]
_[112960]
_[112961]
_[112962]
_[112963]
_[112964]
_[112965]
_[112966]
_[112967]
_[112968]
_[112969]
_[112970]
_[112971]
_[112972]
_[112973]
_[112974]
_[112975]
_[112976]
_[112977]
_[112978]
_[112979]
_[112980]
_[112981]
_[112982]
_[112983]
_[112984]
_[112985]
_[112986]
_[112987]
_[112988]
_[112989]
_[112990]
_[112991]
_[112992]
_[112993]
_[112994]
_[112995]
_[112996]


## Output Analysis

In [4]:
using DataFrames
using CSV

# Define timeslots as a vector
timeslots = ["6-8 PM FRI", "8-10 PM FRI", "8-10 AM SAT", "10 AM-12 PM SAT", 
             "12-2 PM SAT", "2-4 PM SAT", "4-6 PM SAT", "6-8 PM SAT"]

# Ensure name_vals and ind_to_name are defined before this point
# Example:
# name_vals = ["Name1", "Name2", ...]
# ind_to_name = Dict(1 => "Name1", 2 => "Name2", ...)

# Initialize a DataFrame to store the results with name column
results_df = DataFrame(names = name_vals)

# Add timeslot columns and initialize with missing values of type String
for slot in timeslots
    results_df[!, slot] = fill("", length(name_vals))  # Initialize with a String
end

# Initialize sumX to accumulate the sum of variable values
sumX = 0.0

# Iterate through the variables in the JuMP model (example)
for (key, var) in vars
    if value(var) > 0
        sumX += value(var)

        name, time, event, principal = key

        # Populate the DataFrame with the corresponding information
        row_index = findfirst(x -> x == ind_to_name[name], results_df.names)
        if !isnothing(row_index)
            results_df[row_index, timeslots[time]] = ind_to_event[event] * (principal == 1 ? " (Principal)" : "")
        end
    end
end

# Print the first few rows of the DataFrame
println(first(results_df, 5))

# Save the DataFrame to a CSV file
CSV.write("weighted_results.csv", results_df)


5×9 DataFrame
 Row │ names           6-8 PM FRI               8-10 PM FRI              8-10 AM SAT              10 AM-12 PM SAT          12-2 PM SAT              2-4 PM SAT               4-6 PM SAT               6-8 PM SAT              
     │ String31        String                   String                   String                   String                   String                   String                   String                   String                  
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Ivan Ge                                                           Optics (Principal)       Optics (Principal)       Optics (Principal)       Optics (Principal)       Optics (Principal)       Optics (Principal)
   2 │ Daniel Ochoa                             Anatomy and Physiology   Anatomy and Physiology   An

"weighted_results.csv"

In [9]:
sumVals = 0
for var in vars_to_minimize
    sumVals += value(var)
end
println("Sum of vars_to_minimize_pref: ", sumVals)

Sum of vars_to_minimize_pref: 160.0


In [ ]:
print(len(vars_to_minimize_slack_p))

208


In [ ]:
print(vars_to_minimize_slack[-1].X
      )

0.0
